## Lesson 3 Homework
Robert Lisy

- Read dataset
- Calculate the Recency, Frequency, Monitory for each customer (using the pandas method is easier).
- Using K-means algorithm, perform customer transaction segmentation
- Treat each final row as a point observation. Do not user Customer ID or Date fields in the k-means clustering.
    - For k=2, describe how the two centroids of the cluster may describe two different types of transactions.


In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
import seaborn as sns

# Read the data set
file = 'https://library.startlearninglabs.uw.edu/DATASCI420/2019/Datasets/SuperstoreTransaction.csv'
df = pd.read_csv(file)
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [2]:
df.dtypes

Row ID             int64
Order ID          object
Order Date        object
Ship Date         object
Ship Mode         object
Customer ID       object
Customer Name     object
Segment           object
Country           object
City              object
State             object
Postal Code        int64
Region            object
Product ID        object
Category          object
Sub-Category      object
Product Name      object
Sales            float64
Quantity           int64
Discount         float64
Profit           float64
dtype: object

In [3]:
# We need to clean up a bunch of columns
date_cols = ['Order Date', 'Ship Date']
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors='coerce')
    
cat_cols = ['Ship Mode', 'Segment', 'Postal Code', 'Category', 'Sub-Category', 'Region', 'Country', 'City', 'State']
for c in cat_cols:
    df[c] = df[c].astype('category')
    
# No spaces in column names!
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')
    
df.dtypes

Row_ID                    int64
Order_ID                 object
Order_Date       datetime64[ns]
Ship_Date        datetime64[ns]
Ship_Mode              category
Customer_ID              object
Customer_Name            object
Segment                category
Country                category
City                   category
State                  category
Postal_Code            category
Region                 category
Product_ID               object
Category               category
Sub_Category           category
Product_Name             object
Sales                   float64
Quantity                  int64
Discount                float64
Profit                  float64
dtype: object

In [4]:
# Let's look at one customer's activity... pick the most frequent customer.
df['Customer_ID'].value_counts()

WB-21850    37
JL-15835    34
MA-17560    34
PP-18955    34
EH-13765    32
            ..
LD-16855     1
CJ-11875     1
AO-10810     1
RE-19405     1
JR-15700     1
Name: Customer_ID, Length: 793, dtype: int64

In [5]:
df[df['Customer_ID'] == 'WB-21850'].head(40)

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,...,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales,Quantity,Discount,Profit
2133,2134,CA-2015-122210,2015-11-30,2015-12-04,Standard Class,WB-21850,William Brown,Consumer,United States,Philadelphia,...,19134,East,OFF-BI-10003656,Office Supplies,Binders,Fellowes PB200 Plastic Comb Binding Machine,152.991,3,0.7,-122.3928
2134,2135,CA-2015-122210,2015-11-30,2015-12-04,Standard Class,WB-21850,William Brown,Consumer,United States,Philadelphia,...,19134,East,OFF-FA-10000053,Office Supplies,Fasteners,Revere Boxed Rubber Bands by Revere,10.584,7,0.2,-2.3814
2135,2136,CA-2015-122210,2015-11-30,2015-12-04,Standard Class,WB-21850,William Brown,Consumer,United States,Philadelphia,...,19134,East,TEC-PH-10002807,Technology,Phones,Motorla HX550 Universal Bluetooth Headset,94.920,4,0.4,15.8200
2148,2149,CA-2014-141607,2014-12-12,2014-12-17,Standard Class,WB-21850,William Brown,Consumer,United States,Concord,...,94521,West,FUR-FU-10003975,Furniture,Furnishings,Eldon Advantage Chair Mats for Low to Medium P...,43.310,1,0.0,4.3310
3741,3742,CA-2016-137848,2016-01-15,2016-01-21,Standard Class,WB-21850,William Brown,Consumer,United States,New York City,...,10011,East,OFF-EN-10001137,Office Supplies,Envelopes,"#10 Gummed Flap White Envelopes, 100/Box",16.520,4,0.0,7.5992
3742,3743,CA-2016-137848,2016-01-15,2016-01-21,Standard Class,WB-21850,William Brown,Consumer,United States,New York City,...,10011,East,OFF-PA-10004285,Office Supplies,Paper,Xerox 1959,60.120,9,0.0,28.8576
3743,3744,CA-2016-137848,2016-01-15,2016-01-21,Standard Class,WB-21850,William Brown,Consumer,United States,New York City,...,10011,East,OFF-BI-10002225,Office Supplies,Binders,"Square Ring Data Binders, Rigid 75 Pt. Covers,...",49.536,3,0.2,17.3376
4226,4227,CA-2017-120327,2017-11-11,2017-11-16,Standard Class,WB-21850,William Brown,Consumer,United States,Urbandale,...,50322,Central,OFF-FA-10004854,Office Supplies,Fasteners,Vinyl Coated Wire Paper Clips in Organizer Box...,45.920,4,0.0,21.5824
4308,4309,CA-2014-125829,2014-11-04,2014-11-11,Standard Class,WB-21850,William Brown,Consumer,United States,Los Angeles,...,90045,West,TEC-PH-10001079,Technology,Phones,Polycom SoundPoint Pro SE-225 Corded phone,666.344,7,0.2,66.6344
4309,4310,CA-2014-125829,2014-11-04,2014-11-11,Standard Class,WB-21850,William Brown,Consumer,United States,Los Angeles,...,90045,West,FUR-TA-10002041,Furniture,Tables,"Bevis Round Conference Table Top, X-Base",573.728,4,0.2,-64.5444


# Customer Clean Up

You can see from above that each line is a single item on the an order. Customer 'WB-21850' for instance has 3 items on order ID CA-2015-122210. We should GROUP these order lines into a single transaction.

In [6]:
df['Segment'].value_counts()

Consumer       5191
Corporate      3020
Home Office    1783
Name: Segment, dtype: int64

## Transaction based data set

Since we're going to use this to group customers, we don't have to keep ALL of the columns when we roll this up. For instance the `Ship_Date` column is not a property of a customer - it's a property of the logistic system's ability to get product out the door. We'll group by the columns we want to keep, and aggregate UP columns accordingly...

In [7]:
# Create a transaction based data set....
query = """
        SELECT
            Order_ID
            , Order_Date
            , Customer_ID
            , Customer_Name
            , Segment
            , Country
            , City
            , State
            , Region
            , SUM(Sales) AS Total_Sales
            , SUM(Quantity) AS Total_Quantity
            , SUM(Discount) AS Total_Discount
            , SUM(Profit) AS Total_Profit
        FROM df
        GROUP BY 1, 2, 3, 4, 5, 6, 7, 8      
        """
df_trans = ps.sqldf(query, locals())

In [8]:
# Now we can do an RFM calculation, since we have transaction level data.

# First aggregate this up to a transaction date...
daily_agg = df_trans.groupby(['Customer_ID', 'Customer_Name', 'Order_Date']).agg({'Total_Sales': 'sum', 
                                                                                  'Total_Quantity': 'sum',
                                                                                  'Total_Discount': 'sum',
                                                                                  'Total_Profit' : 'sum'})
daily_agg = daily_agg.reset_index()
daily_agg['Order_Date'] = pd.to_datetime(daily_agg['Order_Date'])
daily_agg.head()

,Customer_ID,Customer_Name,Order_Date,Total_Sales,Total_Quantity,Total_Discount,Total_Profit
0,AA-10315,Alex Avila,2014-03-31,726.548,4,0.2,267.4224
1,AA-10315,Alex Avila,2014-09-15,29.500,5,0.0,13.2826
2,AA-10315,Alex Avila,2015-10-04,26.960,2,0.0,7.0096
3,AA-10315,Alex Avila,2016-03-03,4406.072,14,0.8,-747.1021
4,AA-10315,Alex Avila,2017-06-29,374.480,5,0.0,96.5050


In [9]:
# Now calc RFM
roll_sum = daily_agg.groupby('Customer_ID').rolling(window = '7D', on = 'Order_Date')['Total_Sales'].sum()
roll_cnt = daily_agg.groupby('Customer_ID').rolling(window = '7D', on = 'Order_Date')['Total_Quantity'].sum()
roll_min = daily_agg.groupby('Customer_ID')['Order_Date'].diff(periods = 1)

In [10]:
# And smush everything back together
rfm = pd.concat([roll_sum, roll_cnt], axis = 1)
rfm.columns = ['dollar_roll_sum_7D', 'quantity_roll_sum_7D']
rfm = rfm.reset_index()
rfm['last_visit_ndays'] = pd.to_numeric(roll_min.fillna(pd.Timedelta('100 days')).astype('string').str.replace('days', ''))

rfm.head()

,Customer_ID,Order_Date,dollar_roll_sum_7D,quantity_roll_sum_7D,last_visit_ndays
0,AA-10315,2014-03-31,726.548,4.0,100
1,AA-10315,2014-09-15,29.500,5.0,168
2,AA-10315,2015-10-04,26.960,2.0,384
3,AA-10315,2016-03-03,4406.072,14.0,151
4,AA-10315,2017-06-29,374.480,5.0,483


# Clustering

Now that we have RFM for customers, let's cluster them into 2 clusters (per the assignment)


In [11]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Grab only the data we care about for clustering....
cluster_df = rfm.iloc[:, 2:5]

# scale that data so that one variable doesn't outweigh the others...
scaler = StandardScaler()
scaler.fit(cluster_df)
cluster_df_scaled = scaler.transform(cluster_df)

# make the clusters
kmeans = KMeans(n_clusters=2, random_state=0).fit(cluster_df_scaled)
results_df = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_))
results_df.columns=['dollar_roll_sum_7D', 'quantity_roll_sum_7D', 'last_visit_ndays']
results_df.head()

,dollar_roll_sum_7D,quantity_roll_sum_7D,last_visit_ndays
0,237.172974,5.685523,175.104027
1,1659.532203,18.702439,172.547561


# Summary

The 2 clusters (A and B for discussion) look to break down into "good customers" (cluster A) and "poorer customers" (cluster B).

The interesting thing is that good customers and poorer customers look to have about the same time of last visit - 175 and 172 days. However good customers get spend MUCH more money (\\$1,659 vs \\$237) and spend that money on more items (18.7 vs 5.6).